In [1]:
'''
Name   : Rupesh garsondiya
github : @Rupeshgarsondiya
Topic  : Dataset and Dataloader class in pytorch
'''

'\nName   : Rupesh garsondiya\ngithub : @Rupeshgarsondiya\nTopic  : Dataset and Dataloader class in pytorch\n'

In [2]:
# load required library
import os
import torch
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [3]:
# load dataset
path = '/content/Student_CGPA_Pakage.csv'

In [4]:
def read_data(path = path):
    return pd.read_csv(path)

data = read_data()
data = data.drop(columns = ['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7'],axis=1)
data.head()


,CGPA,IQ (*10),Pakage
0,5.0,10.0,3.0
1,8.0,9.0,4.5
2,9.0,9.0,6.0
3,6.0,6.0,3.5
4,7.0,7.5,3.7


In [7]:
# split input feature and traget

X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

print('X shape : ',X.shape)
print('y shape : ',y.shape)

# chnage the typecasting form Numpy array -----> to torch tensor

X = torch.tensor(X,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.float32)

print('X shape : ',X.shape)
print('y shape : ',y.shape)

# split train and test data

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

X shape :  (99, 2)
y shape :  (99,)
X shape :  torch.Size([99, 2])
y shape :  torch.Size([99])


In [9]:
# create the dataloader and dataset classes

# create class called coustomdata

# The CustomData class inherits from the Dataset class, which is already built into PyTorch.
class coustomdata(Dataset):

    '''
    work : inialize the parameter
    '''
    def __init__(self,input_feature,target):
        self.x = input_feature
        self.y = target
    '''
    work : This function returns the length of the dataset.
    '''
    def __len__(self):
        return len(self.x)

    '''
    work : This function returns a single sample from the dataset. In this function we also apply transformation on the our data
    '''
    def __getitem__(self,index):
        return self.x[index],self.y[index]

# create object of the our coustomdata class

train_dataset = coustomdata(X_train,y_train)
test_dataset = coustomdata(X_test,y_test)
# Example: Here, we pass an index number, and it fetches the corresponding row from the dataset.



# Now let's create the object of the dataLoader class

'''In DataLoader constructor we pass out coustomdata class object and batch size and shuffle ,There are also some other pramter
   avilable like no_worker this parameter work to load data parellel
   return : Batch of the data
'''

# Create two dataloader one for the train and one for the testing
train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=4)
test_dataloader = DataLoader(test_dataset,shuffle=False,batch_size=2)

# Now we clearly see that dataloader return the batch of the data
for input,target in test_dataloader:
    print(input)
    print(target)
    print('-'*50)

tensor([[5.2300, 6.7000],
        [8.0000, 8.0000]])
tensor([2.5600, 6.0000])
--------------------------------------------------
tensor([[8.0000, 9.5000],
        [2.6000, 6.0000]])
tensor([4.0000, 3.5000])
--------------------------------------------------
tensor([[10.0000, 15.0000],
        [ 9.6000, 11.5000]])
tensor([12.0000,  5.6000])
--------------------------------------------------
tensor([[5.3600, 6.3000],
        [5.6000, 6.0000]])
tensor([3.0000, 3.5600])
--------------------------------------------------
tensor([[ 3.5000,  9.0000],
        [ 5.0000, 10.0000]])
tensor([2., 3.])
--------------------------------------------------
tensor([[8.5000, 9.5000],
        [9.0000, 9.9000]])
tensor([6.2100, 6.6900])
--------------------------------------------------
tensor([[6.0000, 5.9000],
        [8.0000, 9.6000]])
tensor([3., 6.])
--------------------------------------------------
tensor([[ 8.6300, 10.0000],
        [ 7.0000,  7.5000]])
tensor([8.5000, 3.7000])
---------------------

In [37]:
# now design the neural net

# import nn module

import torch.nn as nn

# class for the neural net
class neuralnet(nn.Module):

    def __init__(self,num_feature):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_feature,16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.BatchNorm1d(8),
            nn.Linear(8,1),
        )
        self.loss = nn.MSELoss()


    def forward(self,x):
        y_pred = self.model(X_train)
        return y_pred


MyNN = neuralnet(X.shape[1])
optimizer = torch.optim.Adam(MyNN.parameters(),lr=0.001)
for i in range(10):

    for input,target in train_dataloader:
        y_pred = MyNN(input)
        loss = MyNN.loss(y_pred,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  # Update parameter
    print(f"Epoch [{i+1}/10], Loss: {loss.item():.4f}")



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([79, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([79, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Loss: 8.3477
Epoch [2/10], Loss: 6.5991
Epoch [3/10], Loss: 28.4170
Epoch [4/10], Loss: 15.8231
Epoch [5/10], Loss: 4.9324
Epoch [6/10], Loss: 7.3898
Epoch [7/10], Loss: 15.0413
Epoch [8/10], Loss: 7.9563
Epoch [9/10], Loss: 8.0607
Epoch [10/10], Loss: 10.9973
